In [1]:
using Flux
using MLDatasets

train_x, train_y = MNIST.traindata(Float32)
test_x, test_y = MNIST.testdata(Float32)

# 入力のベクトル化
train_x = reshape(train_x, 28*28, :)
test_x  = reshape(test_x, 28*28, :)

# 出力のOne-hot Encoding
train_y = onehotbatch(train_y, 0:9)
test_y  = onehotbatch(test_y, 0:9);

# データローダの準備
train_loader = DataLoader((train_x, train_y), batchsize=16, shuffle=true)
test_loader = DataLoader((test_x, test_y), batchsize=16, shuffle=false)

model = Chain(
    Dense(28*28, 100, relu),
    Dense(100, 100, relu),
    Dense(100, 50, relu),
    Dense(50, 10),
    softmax,
)
loss(x,y) = Flux.crossentropy(model(x), y)
optimizer = Flux.ADAM()

┌ Warning: MNIST.traindata() is deprecated, use `MNIST(split=:train)[:]` instead.
└ @ MLDatasets /Users/kizawamasakazu/.julia/packages/MLDatasets/mtOdX/src/datasets/vision/mnist.jl:187
┌ Warning: MNIST.testdata() is deprecated, use `MNIST(split=:test)[:]` instead.
└ @ MLDatasets /Users/kizawamasakazu/.julia/packages/MLDatasets/mtOdX/src/datasets/vision/mnist.jl:195


UndefVarError: UndefVarError: `onehotbatch` not defined

In [5]:
# エポックによる反復学習
epochs = 100

for epoch in 1:epochs
    loss_loss = 0.0
    println("$(epoch)epoch目の学習開始")
    for (i, train_data) in enumerate(train_loader)
        Flux.train!(loss, Flux.params(model), [train_data], optimizer)
        # 途中経過
        (i % 100) == 0 && begin
            loss_value = 0.0
            count = 0
            for (x,y) in test_data_loader
                count += 1
                loss_value += loss(x,y)
            end
            loss_value /= count
        end
    end
    println("epoch=$(epoch)")
end

1epoch目の学習開始
100 損失: 0.528889590281248
200 損失: 0.41280529129207133
300 損失: 0.3332062443003058
400 損失: 0.3028476855017245
500 損失: 0.34030175527483225
600 損失: 0.28135498837903145
700 損失: 0.2564445942506194
800 損失: 0.22800349096246064
900 損失: 0.24485486240871251
1000 損失: 0.23178228713311255
1100 損失: 0.19592455505989492
1200 損失: 0.21540689186351375
1300 損失: 0.22473019910044967
1400 損失: 0.17660146478591487
1500 損失: 0.18526390455150976
1600 損失: 0.1776341816427186
1700 損失: 0.1526700362802483
1800 損失: 0.17441530130351893
1900 損失: 0.1640099706709385
2000 損失: 0.16270766730252653
2100 損失: 0.1372136194191873
2200 損失: 0.1385561321868561
2300 損失: 0.16881504196710884
2400 損失: 0.1427622214773437
2500 損失: 0.17249103755299003
2600 損失: 0.1362716656707227
2700 損失: 0.15004097197027877
2800 損失: 0.1378851543678902
2900 損失: 0.15869457368287257
3000 損失: 0.14904278972195462
3100 損失: 0.14375870103440247
3200 損失: 0.13404894365221262
3300 損失: 0.13579560298905707
3400 損失: 0.12855068596974015
3500 損失: 0.118868822899

In [ ]:
# 検証
test_accuracy = 0.0
cnt = 0
for (i, testdata) in enumerate(testdata_loader)
    x, y = testdata
    outputs = model(x)
    yhat = mapslices(argmax, outputs, dims=1)
    y = mapslices(argmax, y, dims=1)
    test_accuracy += sum(y .== yhat)
    cnt += size(y,2)
end
test_accuracy /= cnt
println("正解率: $(test_accuracy)")